In [2]:
import csv
import re
import os
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

In [47]:
def create_csv_file(filename, train=True):
    
    header = ['row_number', 'text', 'polarity']
    
    if train == True:
        path_to_pos = "datasets/offensive/train/pos/"
        path_to_neg = "datasets/offensive/train/neg/"
    else:
        path_to_pos = "datasets/offensive/test/pos/"
        path_to_neg = "datasets/offensive/test/neg/"
        
    count = 0    
    with open(filename, "w", newline = '') as f1:
         writer = csv.writer(f1, delimiter = ',')
         writer.writerow(header)
         for f in os.listdir(path_to_pos):
            polarity = 1
            if f.endswith(".txt"):
                open_file = open(path_to_pos+f, "r")
                data = open_file.read()
                writer.writerow([count, f'"{data}"', polarity])
                count += 1
                open_file.close()
         for f in os.listdir(path_to_neg):
            polarity = 0
            if f.endswith(".txt"):
                open_file = open(path_to_neg+f, "r")
                data = open_file.read()
                writer.writerow([count, f'"{data}"', polarity])
                count += 1
                open_file.close()

In [48]:
def create_list_of_docs(filename):
    docs = []
    polarity = []
    with open(filename,'r') as csvfile:
            reader = csv.reader(csvfile, delimiter = ',')
            next(reader, None) #this skips the header row
            for row in tqdm(reader):
                docs.append(row[1])
                polarity.append(row[2])    
    return docs, polarity

In [49]:
def remove_special_chars (strs):
    strs = re.sub(r'<.*?>', '', strs)
    strs = re.sub(r'[^a-zA-Z. ]', '', strs) 
    
    return strs

In [50]:
def data_preprocess(docs):
    for index, row in tqdm(enumerate(docs)):
        docs[index] = remove_special_chars(row)    
    return docs

In [ ]:
def ngram_classifier (docs_train, y_train, docs_test, ngram_range, tfidf):
    
    if tfidf == True:
        tfidfvec = TfidfVectorizer(stop_words = "english",
                                  analyzer = 'word',
                                  lowercase = True,
                                  use_idf = True,
                                  ngram_range = ngram_range)
        
        X_train = tfidfvec.fit_transform(docs_train)
    
        X_test = tfidfvec.transform(docs_test)
        
    else:
        cvec = CountVectorizer(stop_words = "english",
                               analyzer = 'word',
                               lowercase = True,
                               ngram_range = ngram_range)
    
        X_train = cvec.fit_transform(docs_train)
        
        X_test = cvec.transform(docs_test)
        
    
    
    clf = SGDClassifier(loss = ",log", penalty = "l1")
    
    clf.fit(X_train, y_train)
    
    prediction = clf.predict(X_test)
    
    return prediction

In [ ]:
y_pred_unigram = ngram_classifier(docs_train,
                                  y_train, 
                                  docs_test, 
                                  (1,1), 
                                  tfidf = False)
                
accuracy_score(y_test_true, y_pred_unigram)